### Installs

In [1]:
!pip install pandas
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann
!pip install ipywidgets

     |████████████████████████████████| 394.7MB 41kB/s 
     |████████████████████████████████| 3.7MB 8.4MB/s 
     |████████████████████████████████| 11.1MB 8.5MB/s 


### Imports

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import math
import heapq
import matplotlib.pyplot as plt

### Data Set-up

In [3]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


In [4]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [5]:
dataframe = tfds.as_dataframe(ratings)
print(dataframe)

       bucketized_user_age     movie_genres  ... user_rating user_zip_code
0                     45.0              [7]  ...         4.0      b'53211'
1                     25.0          [4, 14]  ...         2.0      b'80525'
2                     18.0              [4]  ...         4.0      b'55439'
3                     50.0           [5, 7]  ...         4.0      b'06472'
4                     50.0         [10, 16]  ...         3.0      b'75094'
...                    ...              ...  ...         ...           ...
99995                 25.0       [0, 1, 15]  ...         4.0      b'80027'
99996                 35.0         [13, 16]  ...         4.0      b'60035'
99997                 18.0             [10]  ...         1.0      b'78264'
99998                 35.0  [0, 10, 15, 16]  ...         4.0      b'53210'
99999                 18.0              [4]  ...         2.0      b'95064'

[100000 rows x 12 columns]


In [6]:
# Pull out relevant columns and convert id columns to ints
relevant_cols = dataframe.loc[:, ['user_id', 'movie_id', 'user_rating', 'timestamp']]
relevant_cols['user_id'] = relevant_cols['user_id'].astype(int)
relevant_cols['movie_id'] = relevant_cols['movie_id'].astype(int)
print(relevant_cols)
print(relevant_cols.dtypes)

       user_id  movie_id  user_rating  timestamp
0          138       357          4.0  879024327
1           92       709          2.0  875654590
2          301       412          4.0  882075110
3           60        56          4.0  883326919
4          197       895          3.0  891409199
...        ...       ...          ...        ...
99995      774       228          4.0  888557237
99996      313       333          4.0  891012877
99997      262       567          1.0  879795430
99998      911       183          4.0  892839492
99999      276      1140          2.0  874791894

[100000 rows x 4 columns]
user_id          int64
movie_id         int64
user_rating    float64
timestamp        int64
dtype: object


In [7]:
relevant_cols['latest'] = relevant_cols.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)

train_ratings = relevant_cols[relevant_cols['latest'] != 1]
test_ratings = relevant_cols[relevant_cols['latest'] == 1]

train_ratings = train_ratings[['user_id', 'movie_id', 'user_rating']]
test_ratings = test_ratings[['user_id', 'movie_id', 'user_rating']]

print(train_ratings.shape)
print(test_ratings.shape)

num_users = len(relevant_cols.user_id.unique()) + 1
num_items = len(relevant_cols.movie_id.unique()) + 1

print('Num Users:', num_users)
print('Num Items:', num_items)

(99057, 3)
(943, 3)
Num Users: 944
Num Items: 1683


## Convert to Implicit Feedback Dataset

In [8]:
# Convert rating to 1 for everything to mark that the user has watched this item
train_ratings.loc[:, 'rating'] = 1
print(train_ratings)


       user_id  movie_id  user_rating  rating
0          138       357          4.0       1
1           92       709          2.0       1
2          301       412          4.0       1
3           60        56          4.0       1
4          197       895          3.0       1
...        ...       ...          ...     ...
99995      774       228          4.0       1
99996      313       333          4.0       1
99997      262       567          1.0       1
99998      911       183          4.0       1
99999      276      1140          2.0       1

[99057 rows x 4 columns]


In [9]:
# Add negative samples
all_movies = relevant_cols['movie_id'].unique()

users, items, labels = [], [], []
user_item_set = set(zip(train_ratings['user_id'], train_ratings['movie_id']))
num_negatives = 4

for (u, i) in user_item_set:
  users.append([u])
  items.append([i])
  labels.append([1])
  for _ in range(num_negatives):
    negative_item = np.random.choice(all_movies)
    while (u, negative_item) in user_item_set:
      negative_item = np.random.choice(all_movies)
    users.append([u])
    items.append([negative_item])
    labels.append([0])

## Create Dataset

In [10]:
train_ds = tf.data.Dataset.from_tensor_slices(({'user': users, 'item': items}, labels))
train_ds = train_ds.cache()

In [11]:
train_ds = train_ds.shuffle(1000)
for elem in train_ds.take(10):
  print(elem)

({'user': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([667], dtype=int32)>, 'item': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([702], dtype=int32)>}, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)
({'user': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([130], dtype=int32)>, 'item': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([434], dtype=int32)>}, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)
({'user': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([862], dtype=int32)>, 'item': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([147], dtype=int32)>}, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>)
({'user': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([878], dtype=int32)>, 'item': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([374], dtype=int32)>}, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)
({'user': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([764], dtype=int32)>,

## Define Models

In [12]:
from keras.models import Model
from keras.layers import Embedding, Input, Dense, Reshape, Multiply, Flatten, Lambda, Concatenate
from keras import initializers, regularizers
import sys

def get_GMFmodel(num_users, num_items, latent_dim):
  user_input = Input(shape = (1,), dtype = 'int32', name = 'user')
  item_input = Input(shape = (1,), dtype = 'int32', name = 'item')

  user_embedding = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
  item_embedding = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)

  user_latent = Flatten()(user_embedding(user_input))
  item_latent = Flatten()(item_embedding(item_input))

  prediction_vec = Multiply()([user_latent, item_latent])

  prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(prediction_vec)

  return Model(inputs=[user_input, item_input], outputs=prediction)

In [13]:
def get_MLPmodel(num_users, num_items, latent_dim):
  user_input = Input(shape = (1,), dtype = 'int32', name = 'user')
  item_input = Input(shape = (1,), dtype = 'int32', name = 'item')

  user_embedding = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
  item_embedding = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
  
  user_latent = Flatten()(user_embedding(user_input))
  item_latent = Flatten()(item_embedding(item_input))

  inputs = Concatenate()([user_latent, item_latent])

  layer = Dense(256, activation='relu', name='Layer1', kernel_initializer='glorot_uniform')(inputs)
  layer = Dense(128, activation='relu', name='Layer2')(layer)
  layer = Dense(32, activation='relu', name='Layer3')(layer)
  output = Dense(1, activation='sigmoid', name='Layer4')(layer)

  return Model(inputs=[user_input, item_input], outputs=output)
    

In [14]:
def get_MLPGMFmodel(num_users, num_items, latent_dim):
  user_input = Input(shape = (1,), dtype = 'int32', name = 'user')
  movie_input = Input(shape = (1,), dtype = 'int32', name = 'item')

  user_embedding = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
  movie_embedding = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
  
  user_latent = Flatten()(user_embedding(user_input))
  item_latent = Flatten()(movie_embedding(movie_input))

  #GMF 
  prediction_vec = Multiply()([user_latent, item_latent])
  prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(prediction_vec)

  #MLF
  inputs = Concatenate()([user_latent, item_latent])
  layer = Dense(256, activation='relu', name='Layer1', kernel_initializer='glorot_uniform')(inputs)
  layer = Dense(128, activation='relu', name='Layer2')(layer)
  layer = Dense(32, activation='relu', name='Layer3')(layer)
  output = Dense(1, activation='sigmoid', name='Layer4')(layer)

  predictions = Concatenate()([prediction, output])

  return Model(inputs=[user_input, movie_input], outputs=predictions)


In [15]:
from keras.models import Model
from keras.layers import Embedding, Input, Dense, Reshape, Multiply, Flatten, Lambda, Concatenate, Conv2D, MaxPool2D
from keras import initializers, regularizers
import sys

def get_OuterProductmodel(num_users, num_items, latent_dim):
  user_input = Input(shape = (1,), dtype = 'int32', name = 'user')
  item_input = Input(shape = (1,), dtype = 'int32', name = 'item')

  user_embedding = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
  item_embedding = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)

  user_latent = Flatten()(user_embedding(user_input))
  item_latent = Flatten()(item_embedding(item_input))

  latent_map = tf.linalg.matmul(tf.expand_dims(user_latent, -1), tf.expand_dims(item_latent, 1))

  x = tf.expand_dims(latent_map, -1)

  x = Conv2D(64, 3, activation='relu', padding='same')(x)
  x = MaxPool2D(pool_size=2)(x)
  x = Conv2D(64, 3, activation='relu', padding='same')(x)
  x = Flatten()(x)
  prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(x)

#  prediction_vec = Multiply()([user_latent, item_latent])

#  prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(prediction_vec)

  return Model(inputs=[user_input, item_input], outputs=prediction)

## Evaluation Functions

In [16]:
def evaluateNCDG(ranked_list, target_item):
  for i in range(len(ranked_list)):
    if ranked_list[i] == target_item:
      return math.log(2) / math.log(i + 2)
  
  return 0

In [17]:
def hitRate(item, ranklist):
  for rank in ranklist:
    if item == rank:
      return 1
  return 0

In [18]:
# This method calculates all the evaluation metrics. Individual methods are called from here.
def evaluate(model, testPosRatings, testNegRatings):
    hits = []
    ncdgs = []
    for i in range(len(testPosRatings)):
      hit, ncdg = evaluate_one(model, testPosRatings[i], testNegRatings[i])
      hits.append(hit)
      ncdgs.append(ncdg)

    return hits, ncdg

In [19]:
def evaluate_one(model, posRating, negRatings, N):
  user = posRating[0]
  movie = posRating[1]
  negRatings.append(movie)

  user_input = np.full(len(negRatings)+ 1, user)

  print(user_input)
  print(negRatings)

  predictions = model.predict([user_input, negRatings])

  # associate item with predictions
  items = {}
  for i in range(len(predictions)):
    items[negRatings[i]] = predictions[i]

  rankedList = heapq.nlargest(N, items, items.get)
  ncdg = evaluateNCDG(rankedList.values(), items.values())
  hit = hitRate(item, rankedList)

  return hit, ncdg

### Create 100 Negative Test Points

In [20]:
# Generate 100 Negative Test Examples
# Add negative samples
all_movies = relevant_cols['movie_id'].unique()

users, items, labels = [], [], []
user_item_set = set(zip(train_ratings['user_id'], train_ratings['movie_id']))
user_set = set(train_ratings['user_id'])
num_test_negatives = 100
test_negatives = {}

for u in user_set:
  negatives = []

  for _ in range(num_test_negatives):
    negative_item = np.random.choice(all_movies)
    while (u, negative_item) in user_item_set:
      negative_item = np.random.choice(all_movies)
    
    negatives.append(negative_item)
  
  test_negatives[u] = negatives

print('Number of users:', len(test_negatives))
print('Number of negative points:', len(test_negatives[138]))

Number of users: 943
Number of negative points: 100


In [21]:
def evaluate_model(model):
  hits = []
  ncdgs = []
  N = 10
  completed = 0

  for (u, item) in zip(test_ratings['user_id'], test_ratings['movie_id']):
    negs = test_negatives[u]
    test_negs = np.array([])
    test_users = np.array([])
    items = {}

    for neg in negs:
      test_negs = np.append(test_negs, tf.constant([neg]))
      test_users = np.append(test_users, tf.constant([u]))
    test_negs = np.append(test_negs, tf.constant([item]))
    test_users = np.append(test_users, tf.constant([u]))

    predictions = model.predict({'user': test_users, 'item': test_negs})
    completed += 1

    for i in range(len(predictions)):
      if i == len(predictions) - 1:
        items[item] = predictions[i]
      else:
        items[negs[i]] = predictions[i]

    rankedList = heapq.nlargest(N, items, items.get)

    ncdg = evaluateNCDG(rankedList, item)
    hit = hitRate(item, rankedList)
    
    ncdgs.append(ncdg)
    hits.append(hit)

  
  hit_rate = np.mean(hits)
  mean_ncdg = np.mean(ncdgs)

  return hit_rate, mean_ncdg

In [22]:
from keras.optimizers import Adam

topK = 10

model = get_OuterProductmodel(num_users, num_items, 8)
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
item (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embed (Embedding)          (None, 1, 8)         7552        user[0][0]                       
__________________________________________________________________________________________________
item_embed (Embedding)          (None, 1, 8)         13464       item[0][0]                       
______________________________________________________________________________________________

In [23]:
NUM_EPOCHS = 10
best_hr = 0
best_ncdg = 0
best_epoch = -1
model_path = "OP_model.h5"

for epoch in range(1, NUM_EPOCHS + 1):
  hist = model.fit(train_ds, batch_size=256, epochs=1)

  hit_rate, ncdg = evaluate_model(model)
  print('Epoch', epoch, 'Hit Rate:', hit_rate, 'NCDG:', ncdg)
  model.save(model_path)

  if hit_rate > best_hr:
    best_hr, best_ncdg, best_iter = hit_rate, ncdg, epoch
    model.save(model_path, overwrite=True)

print("Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ncdg))
print("The best Outer Product model is saved to %s" %(model_path))

495285/495285 [==============================] - 1586s 3ms/step - loss: 0.4094
Epoch 1 Hit Rate: 0.432661717921527 NCDG: 0.2361300017082969
495285/495285 [==============================] - 1550s 3ms/step - loss: 0.3947
Epoch 2 Hit Rate: 0.4135737009544008 NCDG: 0.21632572621852292
495285/495285 [==============================] - 1520s 3ms/step - loss: 0.4119
Epoch 3 Hit Rate: 0.32237539766702017 NCDG: 0.1812408771763804
495285/495285 [==============================] - 1533s 3ms/step - loss: 0.4266
Epoch 4 Hit Rate: 0.24284199363732767 NCDG: 0.14425830955984442
495285/495285 [==============================] - 1506s 3ms/step - loss: 0.4458
Epoch 5 Hit Rate: 0.2841993637327678 NCDG: 0.15656416433781958
495285/495285 [==============================] - 1520s 3ms/step - loss: 0.4558
Epoch 6 Hit Rate: 0.2354188759278897 NCDG: 0.13677450665317925
495285/495285 [==============================] - 1511s 3ms/step - loss: 0.4795
Epoch 7 Hit Rate: 0.3425238600212089 NCDG: 0.18402920842354176
495285/

In [31]:
hist2 = model.fit(train_ds, batch_size=256, epochs=10)
x = np.arange(1, 11)
plt.plot(x,hist2.history['binary_accuracy'], '.g-')
plt.plot(x,hist2.history['loss'], '-o')
plt.xticks(x)
plt.title('Accuracy and Loss for Outer Product Model')
plt.legend(['acc', 'loss'])

Epoch 1/10
  3765/495285 [..............................] - ETA: 25:28 - loss: 0.4371

KeyboardInterrupt: ignored

In [ ]:
# # Recreate model with weights and optimizer
# new_model = tf.keras.models.load_model(model_path)

# # Show the model architecture
# new_model.summary()